# Homework 2
## Social Media Analytics

Clarissa Franklin, Kyle Katzen, Paige McKenzie, Meyappan Subbaiah

### Task A.
We brainstormed a list of major political topics that we thought would be relevant from 1789-2018. Our list included eight topics: war, voting, equality, slavery, taxes, economy, transportation, and international relations. We believe that these are the major topics that will appear, but also expected approximately two additional topics to appear in the speeches that we did not include. For this reason, we chose to model 10 topics. 

### Task B. 
#### topic modelling with LDA

In [1]:
import pandas as pd
import os
from nltk.corpus import stopwords
from nltk.stem.porter import *
import nltk
import regex as re
import numpy as np
import gensim
import sklearn
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\clari\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


stop = stopwords.words('english')

        
df = pd.DataFrame(columns=['year', 'month', 'day', 'speaker', 'text'])

for filename in os.listdir("data"):
    with open(os.path.join("data", filename), 'r') as book:
        s = pd.Series(filename.split(' ',1)[0].split('-')+[filename.split(' ',1)[1][:-4], book.read()], 
                      index=['year', 'month', 'day', 'speaker', 'text'])
        df=df.append(s, ignore_index=True)
#test with small
df=df[0:5]

df['text']=df['text'].apply(lambda x: re.sub("[^\w\s'+]",'', x)) 



df['text']=df['text'].apply(lambda x: [stemmer.stem(lemmatizer.lemmatize(item.lower())) for item in x.split() if item.lower() not in stop])

tf_vect=TfidfVectorizer(max_df=0.75, min_df=0.1)

df['text_joined']=df['text'].str.join(sep=" ")
x = tf_vect.fit_transform(df['text_joined']).todense()
tf_words=tf_vect.get_feature_names()

real_words=[]
for word in tf_words:
    first=word.split()[0]
    if first.isalpha():
        real_words+=[word]
real_words

#get frequency of words whole corpus
#init=[]
#corp=[i+init for i in df['text']][0]
#freq_dist = nltk.FreqDist(corp)

#also remove words that do not meet tf-idf thresholds (appear in nearly all documents, or are obscure words that do not appear in more than 10% of documents)
df['text'] = df['text'].apply(lambda x: [ word for word in x if word in real_words])

df.head()

,year,month,day,speaker,text,text_joined
0,1789,04,30,George Washington,"[fellow, citizen, senat, vicissitud, incid, li...",fellow citizen senat hous repres among vicissi...
1,1789,10,03,George Washington,"[wherea, acknowledg, provid, almighti, god, ob...",wherea duti nation acknowledg provid almighti ...
2,1790,01,08,George Washington,"[fellow, citizen, senat, embrac, opportun, con...",fellow citizen senat hous repres embrac great ...
3,1790,12,08,George Washington,"[fellow, citizen, senat, meet, feel, much, abl...",fellow citizen senat hous repres meet feel muc...
4,1790,12,29,George Washington,"[presid, mouth, written, speech, sign, hand, s...",presid unit state mouth written speech sign ha...


In [6]:
from gensim import corpora as cp
#create the term dictionary of our corpus, where every unique term is assigned an index
dictionary=cp.Dictionary(df['text'].values)

len(dictionary)
print(dictionary)
#convert list of documets into document term matrix using the dictionary

doc_term_matrix=[dictionary.doc2bow(doc) for doc in df['text'].values]

Dictionary(1011 unique tokens: ['fellow', 'citizen', 'senat', 'vicissitud', 'incid']...)


In [7]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

#print the topics
topics=pd.DataFrame(ldamodel.print_topics(num_topics=10, num_words=6), columns=['topic', 'words_and_weights'])
topics.head()

,topic,words_and_weights
0,0,"0.001*""measur"" + 0.001*""part"" + 0.001*""citizen..."
1,1,"0.001*""land"" + 0.001*""care"" + 0.001*""futur"" + ..."
2,2,"0.007*""one"" + 0.007*""ought"" + 0.007*""execut"" +..."
3,3,"0.001*""part"" + 0.001*""measur"" + 0.001*""made"" +..."
4,4,"0.001*""land"" + 0.001*""measur"" + 0.001*""citizen..."
